### DP11. 根据目标字段，获取共现矩阵（初步尝试）
[Back to HOME](https://visforever.github.io/)

In [14]:
def generateCSV_string_node(dic):
    s = ''
    for key, value in dic.items():
        s = s + key + ',' + str(value) + '\n'
    return s

In [15]:
# 边太多了，给一个过滤参数（默认：10）
def generateCSV_string_edge(dic, freqNum = 10):
    total = len(dic.keys())
    print("===================================")
    print(f"总数：{total}")
    cnt = 0
    s = ''
    for key, value in dic.items():
        cnt = cnt + 1
        if cnt % 50000 == 0:
            print(f"已处理 / 总数: {cnt} / {total}")

        if value > freqNum:
            s = s + key + ',' + str(value) + '\n'
    return s

In [16]:
def str2csv_node(filePath, s):
    '''
    将字符串写入到本地csv文件中
    :param filePath: csv文件路径
    :param s: 待写入字符串(逗号分隔格式)
    '''
    with open(filePath, 'w', encoding='utf-8') as f:
        f.write("Label,Weight\r")
        f.write(s)
    print('写入文件成功,请在'+filePath+'中查看')

In [17]:
def str2csv_edge(filePath, s):
    '''
    将字符串写入到本地csv文件中
    :param filePath: csv文件路径
    :param s: 待写入字符串(逗号分隔格式)
    '''
    with open(filePath, 'w', encoding='utf-8') as f:
        f.write("Source,Target,Weight\r")
        f.write(s)
    print('写入文件成功,请在'+filePath+'中查看')

In [18]:
import pymongo

def db2csv_string(targetDB, targetCol, freqNum, targetField):

    con_str = "localhost:27017"
    db = pymongo.MongoClient(con_str, serverSelectionTimeoutMS = 10000)
    
    exec(targetDB + " = db['" + targetDB + "']")
    print(targetDB + ": 已连接！")
    
    exec(targetCol + " = " + targetDB + "['" + targetCol + "']")
    exec("recs = " + targetCol + ".find()")
    exec("nums_recs = " + targetCol + ".count_documents({})")
    loc = locals()

    total_count = loc['nums_recs']
    print("===================================")
    print(f"总记录数：{total_count}")


    node_dict = {}  # 节点字典,包含节点名+节点权值(频数)
    edge_dict = {}  # 边字典,包含起点+目标点+边权值(频数)

    pbar = 0
    # 第1层循环,遍历整表的每行信息
    for cW in loc['recs']:
        pbar = pbar + 1
        if pbar % 5000 == 0:
            print(f"已处理记录数 / 总记录数: {pbar} / {total_count}")
        # 手动写词频统计函数，基于字典实现
        row_authors_list = cW.get(targetField)

        # 第2层循环
        for index, pre_au in enumerate(row_authors_list): # 使用enumerate()以获取遍历次数index
            # 统计单个词出现的频次
            if pre_au not in node_dict:
                node_dict[pre_au] = 1
            else:
                node_dict[pre_au] += 1
            # 若遍历到倒数第一个元素,则无需记录关系,结束循环即可
            if pre_au == row_authors_list[-1]:
                break

            connect_list = row_authors_list[index+1:]

            # 第3层循环,遍历当前行词后面所有的词,以统计两两词出现的频次
            for next_au in connect_list:
                A, B = pre_au, next_au
                # 固定两两词的顺序
                # 仅计算上半个矩阵
                if A==B:
                    continue
                if A > B:
                    A, B = B, A
                key = A+','+B  # 格式化为逗号分隔A,B形式,作为字典的键

                # 若该关系不在字典中,则初始化为1,表示词间的共同出现次数
                if key not in edge_dict:
                    edge_dict[key] = 1
                else:
                    edge_dict[key] += 1

    # 对得到的字典按照value进行排序
    # node_str = sortDictValue(node_dict, is_reverse=True)  # 节点
    # edge_str = sortDictValue(edge_dict, is_reverse=True)   # 边
    node_csv_string = generateCSV_string_node(node_dict)
    edge_csv_string = generateCSV_string_edge(edge_dict, freqNum)
    
    
    return node_csv_string, edge_csv_string

In [21]:
if __name__ == '__main__':
    
    # 参数设置
    freqNum = 1
    targetField = 'resTopic'
    
    targetDB = 'db_Weibo_SH0331'
    targetCol = 'col_weibo_timePeriod_0'
    
    #----------------------------------------
    
    filePath1 = f'E:/node_col_{targetField}_{freqNum}.csv'
    filePath2 = f'E:/edge_col_{targetField}_{freqNum}.csv'
    
    print(filePath1)
    print(filePath2)
        


    # 根据共同词列表, 构建共现矩阵
    node_csv_string, edge_csv_string = db2csv_string(targetDB, targetCol, freqNum, targetField)
    print("csv_string generated!")

    # 将字符串写入到本地csv文件中
    str2csv_node(filePath1,node_csv_string)
    print("[node] write done!")
    str2csv_edge(filePath2,edge_csv_string)
    print("[edge] write done!")

E:/node_col_resTopic_1.csv
E:/edge_col_resTopic_1.csv
db_Weibo_SH0331: 已连接！
总记录数：69160
已处理记录数 / 总记录数: 5000 / 69160
已处理记录数 / 总记录数: 10000 / 69160
已处理记录数 / 总记录数: 15000 / 69160
已处理记录数 / 总记录数: 20000 / 69160
已处理记录数 / 总记录数: 25000 / 69160
已处理记录数 / 总记录数: 30000 / 69160
已处理记录数 / 总记录数: 35000 / 69160
已处理记录数 / 总记录数: 40000 / 69160
已处理记录数 / 总记录数: 45000 / 69160
已处理记录数 / 总记录数: 50000 / 69160
已处理记录数 / 总记录数: 55000 / 69160
已处理记录数 / 总记录数: 60000 / 69160
已处理记录数 / 总记录数: 65000 / 69160
总数：4577
csv_string generated!
写入文件成功,请在E:/node_col_resTopic_1.csv中查看
[node] write done!
写入文件成功,请在E:/edge_col_resTopic_1.csv中查看
[edge] write done!
